In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Завантаження та підготовка даних
data = pd.read_csv('heart.csv')
data.columns = ['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS', 'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'Slope', 'Ca', 'Thal', 'HeartDisease']
X = data.drop('HeartDisease', axis=1)
y = data['HeartDisease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Нормалізація даних
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Моделі
log_clf = LogisticRegression(random_state=42)
rf_clf = RandomForestClassifier(random_state=42)
svc_clf = SVC(probability=True, random_state=42)
dt_clf = DecisionTreeClassifier(random_state=42)
knn_clf = KNeighborsClassifier()

# Жорстке голосування
hard_voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rf_clf), ('svc', svc_clf), ('dt', dt_clf)],
    voting='hard')
hard_voting_clf.fit(X_train_scaled, y_train)

# М'яке голосування
soft_voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rf_clf), ('svc', svc_clf), ('dt', dt_clf)],
    voting='soft')
soft_voting_clf.fit(X_train_scaled, y_train)

# Стекінг
estimators = [
    ('rf', rf_clf),
    ('svc', svc_clf),
    ('dt', dt_clf)
]
stacking_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression())
stacking_clf.fit(X_train_scaled, y_train)

# Блендінг тут ви могли б використати змішування моделей, але в Python це не стандартизовано

# Випадковий ліс (Беггінг)
rf_clf.fit(X_train_scaled, y_train)

# Бустінг
gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_clf.fit(X_train_scaled, y_train)

# Перевірка найкращого ансамблю
models = [hard_voting_clf, soft_voting_clf, stacking_clf, rf_clf, gb_clf]
for model in models:
    y_pred = model.predict(X_test_scaled)
    print(f'{model.__class__.__name__}: {accuracy_score(y_test, y_pred)}')

# Визначення важливості ознак для RandomForest
feature_importances = pd.Series(rf_clf.feature_importances_, index=X.columns)
print(feature_importances.sort_values(ascending=False))

VotingClassifier: 0.8688524590163934
VotingClassifier: 0.8852459016393442
StackingClassifier: 0.8688524590163934
RandomForestClassifier: 0.8360655737704918
GradientBoostingClassifier: 0.7868852459016393
Oldpeak           0.128485
MaxHR             0.119725
Ca                0.115533
ChestPainType     0.103792
Thal              0.093300
Age               0.092811
RestingBP         0.077537
ExerciseAngina    0.075809
Cholesterol       0.074812
Slope             0.051058
Sex               0.035658
RestingECG        0.019782
FastingBS         0.011698
dtype: float64
